In [1]:
import torch
import lightning as L
import sys


sys.path.append('../')
sys.path.append('../model')

from classifiers import EncoderClassifier, EncoderClassifierConfig
from training import run_training, LightningWrapper
from encoder import create_model, add_arguments
from esm_train import get_esm, device, parser
from data_loading import prepare_datasets
from torchmetrics import F1Score, MatthewsCorrCoef, Precision, Recall, AUROC, \
MeanMetric, AveragePrecision, PrecisionRecallCurve, MetricCollection
from torch.utils.data import DataLoader
from functools import partial
from data_loading import prep_batch

/storage/praha1/home/nexuso1/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


In [2]:
import numpy as np

In [3]:
add_arguments(parser)

In [3]:
chkpt_path = '../model/new_logs/encoder_S_60_focal/fold_0/chkpt.ckpt'
checkpoint = torch.load(chkpt_path)

In [4]:
args = parser.parse_args(args=[])

for k, v in checkpoint['hyper_parameters'].items():
    args.__setattr__(k, v)
args.batch_size=16
args

NameError: name 'checkpoint' is not defined

In [4]:
def test_model(args, name):
    full_dataset = prepare_datasets(args, ignore_label=args.ignore_label)

    step_metrics = MetricCollection({
        'f1' : F1Score(task='binary', ignore_index=args.ignore_label),
        'precision' : Precision(task='binary',ignore_index=args.ignore_label),
        'recall' : Recall(task='binary', ignore_index=args.ignore_label),
    })

    epoch_metrics = MetricCollection({
        'f1' : F1Score(task='binary', ignore_index=args.ignore_label),
        'precision' : Precision(task='binary',ignore_index=args.ignore_label),
        'recall' : Recall(task='binary', ignore_index=args.ignore_label),
        'auroc' : AUROC('binary', ignore_index=args.ignore_label),
        'auprc' : AveragePrecision('binary', ignore_index=args.ignore_label),
        'mcc' : MatthewsCorrCoef('binary', ignore_index=args.ignore_label)
    })
    
    total_results = {}
    n_folds = 5
    for fold in range(n_folds):
        print(f'Testing fold {fold}')
        chkpt_path = f'../model/new_logs/{name}/fold_{fold}/chkpt.ckpt'
        checkpoint = torch.load(chkpt_path)
        model, tokenizer = create_model(args)
        
        train_ds, dev_ds, test_ds = full_dataset.get_fold(fold)
        train = DataLoader(train_ds, args.batch_size, shuffle=True,
                            collate_fn=partial(prep_batch, tokenizer=tokenizer, ignore_label=args.ignore_label),
                            persistent_workers=True if args.num_workers > 0 else False, 
                            num_workers=args.num_workers )
        dev = DataLoader(dev_ds, args.batch_size, shuffle=False,
                            collate_fn=partial(prep_batch, tokenizer=tokenizer, ignore_label=args.ignore_label),
                            persistent_workers=True if args.num_workers > 0 else False,
                            num_workers=args.num_workers)

        test = DataLoader(test_ds, args.batch_size, shuffle=False,
                            collate_fn=partial(prep_batch, tokenizer=tokenizer, ignore_label=args.ignore_label),
                            persistent_workers=True if args.num_workers > 0 else False,
                            num_workers=args.num_workers)
       
        model = LightningWrapper(args, model, step_metrics=step_metrics, epoch_metrics=epoch_metrics, ds_size=len(train), logdir='.')
        model.load_state_dict(checkpoint['state_dict'])
        model.to(device)
        model = model.classifier
        
        trainer = L.Trainer()
        model.logdir = '.'
        pred_df = {}
        preds = []
        labels = []
        indices = []

        with torch.no_grad():
            for batch in test:
                loss, batch_preds = model.predict(**batch.to(model.device))
                batch_labels = batch['labels']
                mask = batch_labels != model.ignore_index
                ids = test_ds.data.iloc[batch['indices'].cpu().numpy()]['id']
                
                # Save the predictions
                for i, id in enumerate(ids):
                    pred_df[id] = {}
                    pred_df[id]['preds'] = batch_preds[i].squeeze().cpu().numpy()
                    pred_df[id]['mask'] = mask[i].cpu().numpy()
                    pred_df[id]['labels'] = batch_labels[i].cpu().numpy()
                
                preds.extend(batch_preds[mask])
                labels.extend(batch_labels[mask])
                
        #results = trainer.test(model, test)[0]
        results = epoch_metrics(torch.hstack(preds), torch.hstack(labels).to(torch.int8))
        for k, v in results.items():
            if k not in total_results:
                total_results[k] = 0
            
            total_results[k] += v
        total_results[f'fold_{fold}'] = pred_df
            
    for k in results.keys():
        total_results[k] = total_results[k] / n_folds
        print(f'{k}: {total_results[k]}')
    
    return total_results
        

In [50]:
test_results = test_model(args, 'encoder_S_60_focal')

Testing fold 0


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EncoderClassifier(
  (base): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 480, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1026, 480, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-11): 12 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=480, out_features=480, bias=True)
              (key): Linear(in_features=480, out_features=480, bias=True)
              (value): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((480,), eps=1e-05, element

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Testing fold 1


...

Testing fold 4


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EncoderClassifier(
  (base): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 480, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1026, 480, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-11): 12 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=480, out_features=480, bias=True)
              (key): Linear(in_features=480, out_features=480, bias=True)
              (value): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((480,), eps=1e-05, element

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


auprc: 0.6329295039176941
auroc: 0.8434503674507141
f1: 0.5219342708587646
mcc: 0.4823983609676361
precision: 0.6683006882667542
recall: 0.4411296546459198


In [53]:
test_results['auprc']

tensor(0.6329, device='cuda:0')

In [5]:
models = ['encoder_S_60_focal_3layers', 'encoder_T_60_focal_3layers', 'encoder_Y_60_focal_3layers','encoder_Y_60_focal_650M']

In [6]:
model_results = {}
for model in models:
    checkpoint = torch.load(f'../model/new_logs/{model}/fold_{0}/chkpt.ckpt')
    args = parser.parse_args(args=[])

    for k, v in checkpoint['hyper_parameters'].items():
        args.__setattr__(k, v)
    args.batch_size=4
    args
    
    res = test_model(args, model)
    torch.save(res, f'../model/new_logs/{model}_results.pkl')

Testing fold 0


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EncoderClassifier(
  (base): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 480, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1026, 480, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-11): 12 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=480, out_features=480, bias=True)
              (key): Linear(in_features=480, out_features=480, bias=True)
              (value): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((480,), eps=1e-05, element

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:408: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)


Testing fold 1


...

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EncoderClassifier(
  (base): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 1280, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1026, 1280, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-32): 33 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=1280, out_features=1280, bias=True)
              (key): Linear(in_features=1280, out_features=1280, bias=True)
              (value): Linear(in_features=1280, out_features=1280, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=1280, out_features=1280, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((1280,), eps=1e-

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Testing fold 2


FileNotFoundError: [Errno 2] No such file or directory: '../model/new_logs/encoder_Y_60_focal_650M/fold_2/chkpt.ckpt'